# Circle detector exam project

### Python Projektformulering

#### Gruppe: Disgusted Obligation

#### Medlemmer: Jesper Hingelbjerg, Albert Scheel Vandel

Vores ide til et projekt er kameratracking, hvor man skal finde et objekt ud fra et, flere, fokuspunkter og bedømme afstanden, positionen og retningen af objektet, heriblandt når objektet bevæger sig, og sætte det ind i et koordinatsystem. Kameraet vil betragte objektet fra fugleperspektiv. Ideen er at tracke eksempelvis en, på objektet, optegnet cirkel med en viser, radiusstreg. Når cirklen deformeres eller på en eller anden vis skrumper, kan man således se at der er et skift i afstand. Hvis viseren peger i en anden retning, kan man se at der er tale om retningsskift. For at genkende cirklen med viseren vil vi benytte os af object recognition, og vi vil sikkert også bruge noget movement detection når objektet bevæger sig. Desuden kunne vi godt tænke os at mappe en sti ud fra hvor objektet hidtil har været. 

### Vores projekt delt op i tasks:
#### -Kunne finde en cirkel i kameraet.
#### -Kunne finde afstanden af en cirklen ud fra dens størrelse i kameraet.
#### -Kunne tegne en sti efter objektet.
#### -Kunne finde ud af i hvilken retning objektet bevæger sig ved at tracke positionen af objektet.
#### -Kunne finde objektets orientering ud fra viseren.
#### -Kombinere alt dette.


In [ ]:
import numpy as np
import cv2
import sys

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if ret:
        
        #TODO: see points above this cell!!!!
        
        
        cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()